###Retrain


In [ ]:
#!git clone https://github.com/fernandoandreotti/cinc-challenge2017

Cloning into 'cinc-challenge2017'...
remote: Enumerating objects: 632, done.
remote: Total 632 (delta 0), reused 0 (delta 0), pack-reused 632
Receiving objects: 100% (632/632), 20.53 MiB | 5.13 MiB/s, done.
Resolving deltas: 100% (308/308), done.


In [ ]:
ls

cinc-challenge2017/  drive/  sample_data/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#cd to the folder where you put the pkl file and the hdf5 file
%cd /content/drive/My Drive/MLHC/ecg-features/data/files

/content/drive/My Drive/MLHC/ecg-features/data/files


In [ ]:
# Common imports 
import os
import numpy as np
import pandas as pd
#to unzip files: with ZipFile('sampleDir.zip', 'r') as zipObj: zipObj.extractall('temp')
import zipfile as zp
#Visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#gdown is useful to download files from public files in Google Drive. --id is the id of the file
#!gdown --id 1j7Yk0-W9uonQiyyG2B9ZAw5jjZA-UeMS

In [ ]:
import pickle
data_dict = pickle.load(open("challenge2017.pkl","rb"))

In [ ]:
data_dict["data"].shape

(8528,)

In [ ]:
data = data_dict["data"]
labels = data_dict['label']
df = pd.DataFrame(data,columns=["ts"])\
      .assign(label = pd.Series(labels).replace({"N":"1","A":"0","O":"2","~":"3"}))
df['ts'][0].T.shape

(9000,)

In [ ]:
#slide and cut
X=df['ts']
Y=df['label']
window_size=6000
stride=1000
out_X=[]
out_Y=[]
out_pid=[]
n_sample = X.shape[0]
for i in range(n_sample):
    tmp_ts = X[i]
    tmp_Y = Y[i]
    if tmp_Y == '0':
        i_stride = stride
        tmp_Y=0
    elif tmp_Y == '1':
        i_stride = stride//6
        tmp_Y=1
    elif tmp_Y == '2':
        i_stride = stride//2
        tmp_Y=2
    elif tmp_Y == '3':
        i_stride = stride//3
        tmp_Y=3
    for j in range(0, len(tmp_ts)-window_size, i_stride):
        out_X.append(tmp_ts[j:j+window_size])
        out_Y.append(tmp_Y)
        out_pid.append(i)
out_X1=np.array(out_X)
out_Y1=np.array(out_Y)
#out_pid1=np.array(out_pid)



In [ ]:
newdf=pd.DataFrame(columns=['ts','label'])
newdf['ts']=out_X
newdf['label']=out_Y

In [ ]:
#delete unused df to free the RAM
del out_X1,out_Y1,out_X,out_Y

In [ ]:
FS = 300
maxlen = 30*FS

def preprocessing(data):
  #np.concatenate(df["ts"]).ravel().tolist())
  data = np.array([data])
  X = np.zeros((1,maxlen)) #X.shape=(1,6000)
  data = np.nan_to_num(data) # removing NaNs and Infs
  data = data[0,0:maxlen]
  data = data - np.mean(data)
  data = data/np.std(data)
  X[0,:len(data)] = data.T # padding sequence
  data = X
  data = np.expand_dims(data, axis=2) #data.shape=(1,9000) -> data.shape=(1,9000,1)
  return data

newdf['ts']=newdf['ts'].apply(lambda x: preprocessing(x))

#out_X1_=out_X1.apply(lambda x: preprocessing(x))


In [ ]:
from collections import Counter
newX=np.asarray([r[0] for r in newdf['ts']])
newY=newdf['label']
Counter(newY)

In [ ]:
#delete unused df to free the RAM, otherwise Colab crashs
del df
del newdf


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import scipy.io

In [ ]:
model=load_model('ResNet_30s_34lay_16conv.hdf5')
#model=load_model('weights-best.hdf5')
X_train,X_test,y_train,y_test=train_test_split(newX,newY,test_size=0.2,shuffle=True)
del newX
del newY
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.3,shuffle=True)
y_train=to_categorical(y_train)
y_train1=y_train
y_test=to_categorical(y_test)
y_val=to_categorical(y_val)
callbacks = [
                # Early stopping definition
                EarlyStopping(monitor='val_loss', patience=3, verbose=1),           
                # Saving best model
                ModelCheckpoint('weights-best.hdf5', monitor='val_loss', save_best_only=True, verbose=1),
                ]
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=25, batch_size=64,callbacks=callbacks)

Epoch 1/5
533/533 [==============================] - ETA: 0s - loss: 0.3303 - accuracy: 0.8724
Epoch 00001: val_loss improved from inf to 0.31218, saving model to weights-best.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


533/533 [==============================] - 241s 418ms/step - loss: 0.3303 - accuracy: 0.8724 - val_loss: 0.3122 - val_accuracy: 0.8844
Epoch 2/5
533/533 [==============================] - ETA: 0s - loss: 0.3203 - accuracy: 0.8776
Epoch 00002: val_loss improved from 0.31218 to 0.30504, saving model to weights-best.hdf5
533/533 [==============================] - 219s 412ms/step - loss: 0.3203 - accuracy: 0.8776 - val_loss: 0.3050 - val_accuracy: 0.8859
Epoch 3/5
533/533 [==============================] - ETA: 0s - loss: 0.3198 - accuracy: 0.8768
Epoch 00003: val_loss improved from 0.30504 to 0.29013, saving model to weights-best.hdf5
533/533 [==============================] - 220s 413ms/step - loss: 0.3198 - accuracy: 0.8768 - val_loss: 0.2901 - val_accuracy: 0.8925
Epoch 4/5
533/533 [==============================] - ETA: 0s - loss: 0.3117 - accuracy: 0.8796
Epoch 00004: val_loss did not improve from 0.29013
533/533 [==============================] - 238s 446ms/step - loss: 0.3117 - acc

In [ ]:
import gc
gc.collect()

43098

###Prediction

In [ ]:
%cd /content/drive/My Drive/MLHC/ecg-features/data/files

/content/drive/My Drive/MLHC/ecg-features/data/files


In [ ]:
data_dict = pickle.load(open("challenge2017.pkl","rb"))
data = data_dict["data"]
labels = data_dict['label']
df = pd.DataFrame(data,columns=["ts"])\
      .assign(label = pd.Series(labels).replace({"N":'0',"A":'1',"O":'2',"~":'3'}))

In [ ]:
FS = 300
maxlen = 30*FS

def preprocessing(data):
  #np.concatenate(df["ts"]).ravel().tolist())
  data = np.array([data])
  X = np.zeros((1,maxlen)) #X.shape=(1,9000)
  data = np.nan_to_num(data) # removing NaNs and Infs
  data = data[0,0:maxlen]
  data = data - np.mean(data)
  data = data/np.std(data)
  X[0,:len(data)] = data.T # padding sequence X[0,:len(data)], len(data)<=9000
  data = X
  data = np.expand_dims(data, axis=2) #data.shape=(1,9000) -> data.shape=(1,9000,1)
  return data

df['ts']=df['ts'].apply(lambda x: preprocessing(x))

#out_X1_=out_X1.apply(lambda x: preprocessing(x))

In [ ]:
print(df['label'][0:5])
to_categorical(df['label'])[0:5]

0    0
1    0
2    0
3    1
4    1
Name: label, dtype: object


array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
from keras.models import load_model
model = load_model('weights-best.hdf5')

#classes = ['0','1','2','3']
classes = ['Normal','AF','Other','Noise']
confusion_matrix = {
    '0': [0, 0, 0, 0],
    '1': [0, 0, 0, 0],
    '2': [0, 0, 0, 0],
    '3': [0, 0, 0, 0]
    
}
#res=[]
for i in range(len(df)):   
  prob = model.predict(df["ts"][i])
  ann = np.argmax(prob)
  true_target = df["label"][i]
  #res.append((ann,true_target))
  #print(true_target,ann)

  confusion_matrix[true_target][ann] += 1

  #print("Record {} classified as {} with {:3.1f}% certainty. True: {}".format(i,classes[ann],100*prob[0,ann],df["label"][i]))


confusion_matrix = pd.DataFrame(confusion_matrix,index = classes)
confusion_matrix.columns = [i + " T" for i in classes]
print(confusion_matrix)

In [ ]:
accuracy = sum([confusion_matrix.iloc[i,i] for i in range(4)])/len(df)
print(f"Total accuracy of {round(accuracy*100,2)}%")

Total accuracy of 84.09%


In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

st.write("""
# Simple Iris Flower Prediction App
This app predicts the **Iris flower** type!
""")

st.sidebar.header('User Input Parameters')

def user_input_features():
    sepal_length = st.sidebar.slider('Sepal length', 4.3, 7.9, 5.4)
    sepal_width = st.sidebar.slider('Sepal width', 2.0, 4.4, 3.4)
    petal_length = st.sidebar.slider('Petal length', 1.0, 6.9, 1.3)
    petal_width = st.sidebar.slider('Petal width', 0.1, 2.5, 0.2)
    data = {'sepal_length': sepal_length,
            'sepal_width': sepal_width,
            'petal_length': petal_length,
            'petal_width': petal_width}
    features = pd.DataFrame(data, index=[0])
    return features

df = user_input_features()

st.subheader('User Input parameters')
st.write(df)

iris = datasets.load_iris()
X = iris.data
Y = iris.target

clf = RandomForestClassifier()
clf.fit(X, Y)

prediction = clf.predict(df)
prediction_proba = clf.predict_proba(df)

st.subheader('Class labels and their corresponding index number')
st.write(iris.target_names)

st.subheader('Prediction')
st.write(iris.target_names[prediction])
#st.write(prediction)

st.subheader('Prediction Probability')
st.write(prediction_proba)